In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Langchain configuration
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

# OpenAI API Key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

# Built-in Functions

The built-in chains, `create_stuff_documents_chain` and `create_retrieval_chain` allow us to craft a retrieval pipeline with following ingredients:

- retriever: This can be created from a vector store
- prompt: This is the query and the placeholder for the context
- LLM: This is the language model that will be used to answer the question based on the context


In [2]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=["https://lilianweng.github.io/posts/2023-06-23-agent/"],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# Split the documents into chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create a retriever from the documents.
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vector_store.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create the model
llm = ChatOpenAI()

# Create the chains
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

'Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable. By decomposing a task, an agent or model can utilize more computational resources and effectively plan ahead by thinking step by step. Techniques like Chain of Thought (CoT) prompt models to decompose hard tasks into multiple manageable tasks, enhancing performance on complex tasks.'

In [4]:
response = rag_chain.invoke({"input": "What are common ways of doing it?"})
response["answer"]

'Common ways of task decomposition include using LLM with simple prompting like "Steps for XYZ" or asking for subgoals, providing task-specific instructions such as "Write a story outline," or involving human inputs in the decomposition process. Additionally, integrating reasoning and acting within LLM, as seen in ReAct, allows for a combination of task-specific actions and natural language reasoning traces to be generated. Lastly, utilizing tool APIs and functions within LLMs, such as in ChatGPT Plugins and OpenAI API function calling, showcases the practical application of tool use capabilities in various tasks.'

In [5]:
response = rag_chain.invoke({"input": "What is the best burger joint in New York?"})
response["answer"]

"I don't know."

# History

> Human: "What is Task Decomposition?"
>
> AI: "Task decomposition involves breaking down complex tasks into smaller and simpler steps to make them more manageable for an agent or model."
>
> Human: "What are common ways of doing it?"

To answer the second question, the AI needs to know what `it` refers to. So we need to add two things:

- Prompt: Update our prompt to support historical messages as an input
- Contextualizing questions: Add a sub-chain that takes the latest user question and reformulates it in the context of the chat history. That is, the pipeline will be changed:

Before:
query -> retriever
After:
(query, conversation history) -> LLM -> rephrased query -> retriever


In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [8]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Task decomposition can be achieved through various methods, such as using Chain of Thought (CoT) where models are instructed to "think step by step" to decompose tasks into smaller steps. Another common approach is by utilizing Language Model (LLM) with simple prompting or task-specific instructions to break down tasks into manageable subgoals. Additionally, human inputs can also be used for task decomposition to provide additional context and guidance.


#
